In [1]:
cd ..

C:\Users\johan\github\designresolver


In [2]:
from polycasebuilder import generate_random_polynomials, get_arg_mapping, directed_poly_executables, eqv_to_edges_tree
from testproblems import generate_random_prob
from graphutils import flat_graph_formulation, draw_graph_graphical_props, intermediary_variables, edges_to_Ein_Eout, Node, VAR, COMP, SOLVER, solver_children
from nestedgraph import build_typedgraph
from operators import sort_scc, reorder_merge_solve
from workflow import get_f, generate_workflow, implicit_comp_name
from assembly import build_archi
from executeformulations import extractvals, generate_x0, run_and_save_archi
import pandas as pd
from representations import bipartite_repr
from inputresolver import resolve,invert_edges,getallvars,direct_eqs

In [12]:
n_eqs = 10
n_vars = 13
seed = 42#8 is triang#seed 10 is nice
sparsity = 1.3#0.8 1.1 #1.7
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed, sparsity)
polynomials, var_mapping, edges, tree, components = generate_random_polynomials(eqv, dout, n_eqs)
#symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

KeyError: 'x_0'

In [10]:
from graphutils import all_variables
from compute import Var
import numpy as np
output_set = dout
output_set = {key:var-n_eqs for key,var in output_set.items()}
edges, tree= eqv_to_edges_tree(eqv, output_set, n_eqs)
Ein, Eout, _ = edges
all_vars = all_variables(Ein,Eout)
var_mapping = {idx: (Var(str(Node(idx,VAR)), varid=idx), str(Node(idx,VAR))) for idx in all_vars}
rng = np.random.default_rng(12345)
arg_mapping = get_arg_mapping(var_mapping)

In [11]:
arg_mapping

{0: 'x_0',
 1: 'x_1',
 2: 'x_2',
 3: 'x_3',
 4: 'x_4',
 5: 'x_5',
 6: 'x_6',
 7: 'x_7',
 8: 'x_8',
 9: 'x_9',
 10: 'x_10',
 11: 'x_11',
 12: 'x_12'}

In [5]:
eqv

{0: (13, 14, 16, 17, 19),
 1: (10, 15, 11),
 2: (11, 12, 15, 16, 17, 20, 21, 22, 14),
 3: (14, 18, 19, 10),
 4: (10, 17, 18, 12),
 5: (12, 17, 19, 20, 13),
 6: (20, 22, 17),
 7: (10, 11, 19, 15),
 8: (10, 17, 18),
 9: (10, 13, 20)}

Optimize/presolve structure

In [4]:
eqs=direct_eqs(eqv, dout)
_, edges_original = bipartite_repr(eqv)
avrs = getallvars(eqs, sympy=False)
eqns = eqs.keys()
graph_edges = invert_edges(edges_original)

In [5]:
output = resolve(eqns, avrs, graph_edges, 50)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-25


In [6]:
soutput = sorted(output, key=lambda x: x['CLEN'])

In [7]:
soutput[0]['CLEN']

[2.0]

In [8]:
out_opt = {eq:var-n_eqs for eq,var in soutput[0]['SOL']}

In [9]:
edges_opt, tree_opt = eqv_to_edges_tree(eqv, out_opt, n_eqs)

In [10]:
new_components = directed_poly_executables(var_mapping, polynomials, out_opt)

In [11]:
components += new_components

In [129]:
components

[((7, 4, 6, 9, 3), 0, (None,), '-0.3*x_3*x_7*x_9 + 0.2*x_4*x_6 - 0.6'),
 ((5, 1, 0), 1, (None,), '0.9*x_0*x_1*x_5 - 0.5'),
 ((5, 10, 11, 2, 12, 7, 4, 1, 6), 2, (None,), '-0.3*x_1*x_11*x_4 - 0.7*x_10 - 0.8*x_12*x_2*x_6 - 0.6*x_5*x_7 - 0.3'),
 ((8, 4, 0, 9), 3, (None,), '-0.6*x_0*x_8 + 0.6*x_4*x_9 - 0.7'),
 ((7, 8, 0, 2), 4, (None,), '0.7*x_0*x_2*x_8 + 0.2*x_7 + 0.9'),
 ((7, 10, 9, 2, 3), 5, (None,), '0.1*x_10*x_2*x_3 + 0.9*x_7*x_9 - 0.5'),
 ((7, 12, 10), 6, (None,), '-0.3*x_10*x_12*x_7 + 0.8'),
 ((5, 1, 9, 0), 7, (None,), '-0.5*x_0*x_1 - 0.3*x_5*x_9 - 1.0'),
 ((7, 8, 0), 8, (None,), '-0.4*x_0*x_7*x_8 - 0.9'),
 ((10, 3, 0), 9, (None,), '-0.1*x_0 - 0.4*x_10*x_3 - 0.7'),
 ((7, 4, 6, 3), 0, (9,), '(0.666666666666667*x_4*x_6 - 2.0)/(x_3*x_7)'),
 ((5, 0), 1, (1,), '0.555555555555556/(x_0*x_5)'),
 ((5, 11, 10, 2, 12, 7, 1, 6), 2, (4,), '-(2.33333333333333*x_10 + 2.66666666666667*x_12*x_2*x_6 + 2.0*x_5*x_7 + 1.0)/(x_1*x_11)'),
 ((4, 8, 9), 3, (0,), '(x_4*x_9 - 1.16666666666667)/x_8'),
 ((7, 8, 

In [12]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

# Find sources of a nested tree

In [13]:
from collections import defaultdict

In [17]:
def dfs_tree(tree, branch):
    childrenmap = defaultdict(list)
    for key,val in tree.items():
        childrenmap[val].append(key)
    visited = set()
    q = [branch]
    while q:
        elt = q.pop()
        visited.add(elt)
        q.extend(childrenmap.get(elt, []))
    return visited

def nested_sources(edges, trees, branch):
    Ein,Eout = edges_to_Ein_Eout(edges)
    Ftree,_,Vtree=trees
    descendants = dfs_tree(Vtree, branch)
    inputs_strictly_below = solver_children(Vtree, descendants-{branch}, solverlist=True)
    comps_below = solver_children(Ftree, descendants-{branch}, solverlist=True)
    srcs = sources(Ein, Eout, filterto=comps_below)
    return srcs - set(inputs_strictly_below)

In [18]:
Stree = {
    5: 3,
    2: 1,
    3: 2,
    4: 2,
    6: 1
}

In [19]:
dfs_tree(Stree, 1)

{1, 2, 3, 4, 5, 6}

Added presolve structure
# Build workflow

In [20]:
def get_solver_implicit_system(groups, tree, solver_idx):
    s = groups[str(Node(solver_idx, SOLVER))]
    name = implicit_comp_name((Node(idx, COMP) for idx in solver_children(tree[0], solver_idx)))
    return getattr(s, name)

In [25]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
lookup_f = get_f(components, nedges)
workflow = generate_workflow(lookup_f, nedges, ntree, {2: {'maxiter':100}})
prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
solver_idx = 2
impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]

In [27]:
scc_guess_vars

['x_10', 'x_2', 'x_9', 'x_5', 'x_4', 'x_8', 'x_0', 'x_3', 'x_7', 'x_1']

In [51]:
import numpy as np
def generate_x0(varnames=None, optres=None, rand_range=(10,20), fixed_direction=None):
    direction = lambda : fixed_direction if fixed_direction else np.random.choice([-1,1]) 
    varnames = optres.keys() if optres else varnames
    x0 = {name:(optres.get(name, None) if optres else 0.1)+np.random.uniform(*rand_range)*direction() for name in varnames}
    return x0

In [75]:
def perturb_inputs(root_ins, root_rand_range, solver_ins, solver_rand_range, xref=None):
    if xref:
        xref = dict(xref) # copy for immutability
        x0_solvers = {key:xref[key] for key in solver_ins} 
        x0_solvers = generate_x0(optres=x0_solvers, rand_range=solver_rand_range)
    else:
        xref = generate_x0(root_ins, rand_range=root_rand_range)
        x0_solvers = generate_x0(solver_ins, rand_range=solver_rand_range)
    xref.update(x0_solvers)
    return xref

In [76]:
x0 = {'x0': 1, 'x1': 1, 'x2':1, 'x3':1}

In [77]:
perturb_inputs(['x0','x3'], (0,0), ['x2','x1'], (0,10), x0)

{'x0': 1, 'x1': 4.5968487321092955, 'x2': -7.597068869881916, 'x3': 1}

In [117]:
def run_full(edges, tree, components, xref=None):
    optres_all = []
    solver_idx = 2
    G = flat_graph_formulation(*edges)
    merge_order = sort_scc(G)
    nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
    lookup_f = get_f(components, nedges)
    workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})
    prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
    impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
    scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]
    prob.setup();
    root_rand_range = (0,0)
    solver_rand_range = (0,1)
    x0_in = perturb_inputs(mdao_in, root_rand_range, scc_guess_vars, solver_rand_range, xref)
    print(x0_in)
    optres, optres_save = run_and_save_archi(prob, x0_in, symb_mapping)
    optres_save.update({'count': impl_system.iter_count_apply}) 
    optres_all.append(optres_save)
    return optres, x0_in, prob

In [118]:
optres, x0_in, prob = run_full(edges, tree, components)

{'x_11': 0.1, 'x_6': 0.1, 'x_12': 0.1, 'x_10': -0.3866379968583621, 'x_2': 0.2877212286612516, 'x_9': 0.7494588832440994, 'x_5': 0.5413092228959531, 'x_4': -0.6643726086611824, 'x_8': 0.3740864619922246, 'x_0': 0.2190953574782604, 'x_3': 0.7287879088794833, 'x_7': 0.5921359986061542, 'x_1': -0.16581911753550724}

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 56 iterations
NL: Newton Converged in 2 iterations


# Checking results

In [119]:
outs = extractvals(prob, get_arg_mapping(var_mapping, symbol_map=True))

In [120]:
comps_res = [comp for comp in components if comp.outputs == (None,)]

In [121]:
[elt.evaldict(outs) for elt in comps_res]

[2.6210467130027837e-10,
 -3.426996020294837e-09,
 1.7908147187384316e-10,
 2.4409152477034013e-09,
 -1.7872260449358635e-08,
 7.86056664203727e-10,
 2.1685986340003183e-12,
 -6.848480338383922e-09,
 -1.6916194001126428e-08,
 2.510305296965498e-10]

In [122]:
x0_in

{'x_11': 0.1,
 'x_6': 0.1,
 'x_12': 0.1,
 'x_10': -0.3866379968583621,
 'x_2': 0.2877212286612516,
 'x_9': 0.7494588832440994,
 'x_5': 0.5413092228959531,
 'x_4': -0.6643726086611824,
 'x_8': 0.3740864619922246,
 'x_0': 0.2190953574782604,
 'x_3': 0.7287879088794833,
 'x_7': 0.5921359986061542,
 'x_1': -0.16581911753550724}

In [123]:
optres

{'x_0': -13.226201359630176,
 'x_1': 0.15067652764667222,
 'x_2': 5.475009162529712,
 'x_3': -0.5364875426161012,
 'x_4': 33.149295768146935,
 'x_5': -0.27877046428001967,
 'x_6': 0.1,
 'x_7': -9.191051589338105,
 'x_8': -0.018508966777741397,
 'x_9': 0.042579184854124395,
 'x_10': -2.9013727545092345,
 'x_11': 0.1,
 'x_12': 0.1}

In [126]:
optres2, x0_in2, prob2 = run_full(edges_opt, tree_opt, components, optres)

{'x_0': -13.226201359630176, 'x_1': 0.15067652764667222, 'x_2': 5.475009162529712, 'x_3': -0.6263085803537081, 'x_4': 33.149295768146935, 'x_5': -0.27877046428001967, 'x_6': 0.1, 'x_7': -9.191051589338105, 'x_8': -0.018508966777741397, 'x_9': 0.042579184854124395, 'x_10': -3.819983662447156, 'x_11': 0.1, 'x_12': 0.1}

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
NL: Newton Converged in 2 iterations


In [64]:
def run_full(edges, tree, components, x0_in=None):
    optres_all = []
    solver_idx = 2
    G = flat_graph_formulation(*edges)
    merge_order = sort_scc(G)
    nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
    lookup_f = get_f(components, nedges)
    workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})
    prob, mdao_in, groups = build_archi(nedges, ntree, workflow)
    impl_system = get_solver_implicit_system(groups, ntree, solver_idx) # we use this for counting
    scc_guess_vars = [str(Node(vr, VAR)) for vr in solver_children(ntree[2], solver_idx)]
    prob.setup();
    rand_range = (0,1)
    if not x0_in:
        x0_in = generate_x0(mdao_in, rand_range=(0,0))
    for i in range(5):
        x0 = dict(x0_in)
        #x0_scc = generate_x0(scc_guess_vars, rand_range=rand_range)
        #x0.update(x0_scc) # this could override x0 ins from scc but thats ok
        #print(x0)
        optres, optres_save = run_and_save_archi(prob, x0, symb_mapping)
        optres_save.update({'count': impl_system.iter_count_apply}) 
        optres_all.append(optres_save)
    return optres_all

In [62]:
pd.DataFrame.from_dict(results_default_case)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,count
0,-13.226201,0.150677,5.475009,-0.536488,33.149296,-0.278770,0.1,-9.191052,-0.018509,0.042579,-2.901373,0.1,0.1,262
1,-13.226205,0.150677,5.475003,-0.536488,33.149298,-0.278770,0.1,-9.191051,-0.018509,0.042579,-2.901373,0.1,0.1,181
2,46.392218,-0.000417,-0.454694,0.516928,29.078760,-28.693481,0.1,-1.032716,0.046963,0.115046,-25.821888,0.1,0.1,119
3,0.113836,-16.977890,13.821738,-0.561735,2.305145,-0.287451,0.1,8.422801,-2.346637,0.390229,3.166009,0.1,0.1,125
4,0.113836,-16.977890,13.821738,-0.561735,2.305145,-0.287451,0.1,8.422801,-2.346637,0.390229,3.166009,0.1,0.1,34


In [61]:
results_default_case = run_full(edges, tree, components)
x0 = {key:var for key,var in results_default_case[0].items() if key != 'count'}

{'x_12': 0.1, 'x_11': 0.1, 'x_6': 0.1, 'x_10': -0.3866379968583621, 'x_2': 0.2877212286612516, 'x_9': 0.7494588832440994, 'x_5': 0.5413092228959531, 'x_4': -0.6643726086611824, 'x_8': 0.3740864619922246, 'x_0': 0.2190953574782604, 'x_3': 0.7287879088794833, 'x_7': 0.5921359986061542, 'x_1': -0.16581911753550724}

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 56 iterations
NL: Newton Converged in 2 iterations
{'x_12': 0.1, 'x_11': 0.1, 'x_6': 0.1, 'x_10': 0.006636966365663716, 'x_2': -0.2275639476887341, 'x_9': 0.9924110171617034, 'x_5': 1.063930529067942, 'x_4': 0.009451304462442939, 'x_8': 0.12430656162948697, 'x_0': 0.784915607421425, 'x_3': -0.6795145855555298, 'x_7': -0.46318750662850483, 'x_1': 0.101

In [65]:
results_opt_case = run_full(edges_opt, tree_opt, components, x0_in=x0)


===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: NewtonSolver 'NL: Newton' on system 's_1.s_2' failed to converge in 100 iterations.
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 64 iterations
NL: Newton Co

In [43]:
results_default_case[0]

{'x_0': 0.008958106779525476,
 'x_1': -13.145086033480357,
 'x_2': 0.7424238083735113,
 'x_3': 12.148139891511779,
 'x_4': -1.4592582797318507,
 'x_5': -4.717890176820716,
 'x_6': -10.79821037737607,
 'x_7': 1.0528907779309629,
 'x_8': -238.5519447614448,
 'x_9': 0.6649317262384284,
 'x_10': -0.14423932736560077,
 'x_11': -19.086109079379213,
 'x_12': -17.559078564803155,
 'count': 106}

In [41]:
pd.DataFrame.from_dict(results_default_case)

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,count
0,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,106
1,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
2,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
3,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2
4,0.008958,-13.145086,0.742424,12.14814,-1.459258,-4.71789,-10.79821,1.052891,-238.551945,0.664932,-0.144239,-19.086109,-17.559079,2


In [16]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)

In [17]:
merge_order

[{f_1, f_3, f_4, f_7, f_8, f_9}, {f_5}, {f_2}, {f_6}, {f_0}]

In [6]:
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)

In [7]:
lookup_f = get_f(components, nedges)

In [8]:
workflow = generate_workflow(lookup_f, nedges, ntree, {2:{'maxiter':100}})

In [9]:
prob, mdao_in, groups = build_archi(nedges, ntree, workflow)

In [10]:
prob.setup();

In [11]:
symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

In [12]:
scc_guess_vars = [str(Node(vr, VAR)) for vr in ntree[2].keys()]

In [43]:
x0_in = generate_x0(mdao_in, rand_range=(10,20))

In [48]:
rand_range = (100,200)
optres_all=[]
for i in range(5):
    x0_scc = generate_x0(scc_guess_vars, rand_range=rand_range)
    optres, optres_save = run_and_save_archi(prob, {**x0_in, **x0_scc}, symb_mapping)
    optres_save.update({'count': impl_system.iter_count_apply}) 
    optres_all.append(optres_save)
        #x0 = generate_x0(optres=optres, rand_range=rand_range)


===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 8 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 32 iterations
NL: Newton Converged in 0 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 38 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 26 iterations
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 0 iterations
NL: Newton Converged in 1 iterations

===
s_1
===
+  
+  =======
+  s_1.s_2
+  =======
+  NL: Newton Converged in 50 iterations
NL: Newton Converged in 0 iterations
